In [ ]:
#Downloading the data
import zipfile
import urllib.request

url = "https://files.grouplens.org/datasets/movielens/ml-100k.zip"
urllib.request.urlretrieve(url, "ml-100k.zip")

with zipfile.ZipFile("ml-100k.zip", "r") as zip_ref:
    zip_ref.extractall("data/raw")


In [28]:
%load_ext autoreload
%autoreload 2


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [29]:
from data_loader import MoviesLensLoader
import pandas as pd

In [30]:

loader = MoviesLensLoader()
ratings, items = loader.load_all()
display(ratings.sample(5))
print("-"*50)
display(items.sample(5))

,user_id,item_id,rating,timestamp
29722,339,737,3,891035180
47741,478,616,4,889398260
2559,200,241,4,884129782
87846,534,824,4,877808260
97900,747,305,5,888638183


--------------------------------------------------


,item_id,title,release_date,video_release,imdb_url,genre_0,genre_1,genre_2,genre_3,genre_4,...,genre_9,genre_10,genre_11,genre_12,genre_13,genre_14,genre_15,genre_16,genre_17,genre_18
1575,1576,"Hungarian Fairy Tale, A (1987)",01-Jan-1987,NaN,"http://us.imdb.com/M/title-exact?Hol%20volt,%2...",0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
306,307,"Devil's Advocate, The (1997)",01-Jan-1997,NaN,http://us.imdb.com/M/title-exact?Devil's+Advoc...,0,0,0,0,0,...,0,0,1,0,1,0,0,1,0,0
850,851,Two or Three Things I Know About Her (1966),01-Jan-1966,NaN,http://us.imdb.com/M/title-exact?Deux%20ou%20t...,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
222,223,Sling Blade (1996),22-Nov-1996,NaN,http://us.imdb.com/M/title-exact?Sling%20Blade...,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
382,383,"Flintstones, The (1994)",01-Jan-1994,NaN,"http://us.imdb.com/M/title-exact?Flintstones,%...",0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
